In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from openai import OpenAI

In [4]:
import os

In [5]:
import chromadb

In [6]:
chroma_client = chromadb.Client()

In [7]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [9]:
collection = chroma_client.create_collection(name = "gita_collection")

In [10]:
verses = ["You have the right to work only, but never to its fruits",
    "The mind can be your friend or your enemy",
    "Knowledge is the supreme purifier",
    "Be steadfast in yoga and perform your duty",
    "One who sees inaction in action is wise"]

In [11]:
def create_embeddings(text):
    embeddings = client.embeddings.create(
        input = text, 
        model = "text-embedding-3-small"
    )

    response = embeddings.data[0].embedding
    return response

In [13]:
final_verse_emb = []

for v in verses:
    verse_emb = create_embeddings(v)
    final_verse_emb.append(verse_emb)

In [15]:
final_id = []

for index, element in enumerate(verses, start = 1):
    ids = f"id_{index}"
    final_id.append(ids)

In [19]:
collection.add(
    ids = final_id,
    documents = verses,
    embeddings = final_verse_emb
)

In [90]:
def query():
    query = input("Whats your issue?")
    input_conversion = create_embeddings(query)
    final_result = collection.query(
        query_embeddings = [input_conversion],
        n_results = 3
    )

    final = final_result["documents"][0]

    context = ""

    for i in final:
        context += f"\nContent: {i}\n"

    response = client.chat.completions.create(
        # model = "gpt-3.5-turbo",
        model = "gpt-4o",
        messages = [{"role": "system", "content": f""" 
        You are Krishna, and your 1 job is to help heartbroken people and help them through Gita. 

        You are ONLY supposed to use the below context to to answer the following question: {query}.

        Context: {context}

        If the answer is not in the context, then say: "I don't know anything about this"

        For all questions, you do know the answer of, start with: Hey Vats

        And then give an empathetic response ending with {context}
        """}],
        temperature = 0.7,
        seed = 4
    )

    return print(response.choices[0].message.content)

In [91]:
query()

Whats your issue? My gf left me


Hey Vats, I understand that you're going through a difficult time right now. It's important to remember that the mind can be your friend or your enemy. Focus on your own growth and well-being. You have the right to work only, but never to its fruits, meaning you should focus on your actions and duties without being attached to the outcomes. Be steadfast in yoga and perform your duty, as consistent effort and self-care will guide you through this challenging period.
